In [2]:
import pandas as pd 

# read the raw dataset and display few rows
raw_data = pd.read_csv('conversion_project.csv')

#display(raw_data)
raw_data.head(5)

,country,age,new_user,source,total_pages_visited,converted
0,UK,25,1,Ads,1,0
1,US,23,1,Seo,5,0
2,US,28,1,Seo,4,0
3,China,39,1,Seo,5,0
4,US,30,1,Seo,6,0
